In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Imports and installations**

In [2]:
!pip install transformers requests

In [3]:
import requests
import json
import transformers


## **Get dataset**

In [4]:
# import the figma file data
# FOR REFERENCE CHECK THE FORMAT HERE
# - file id: Identifier for the figma file
# - access token: Required for authentication with the figma api

def fetch_figma_data(file_id,token):
  url= f"https://api.figma.com/v1/files/{file_id}"

  # setting up headers for the http requests
  headers = {"X-Figma-Token": token}

  # http GET request to the figma api to get the file data
  response = requests.get(url,headers=headers)
  return response.json()

In [5]:
# use the method
file_id= "vyKyzo7v71QHiJPbi3vlmB"
access_token= "figd_H1VkRVA7rbkAGd10Xf22Ta_UgDAX6wRvXC4foy3I"
figma_data= fetch_figma_data(file_id, access_token)
# print the fetched Figma file data to the console
#print(file_data)

In [6]:
def count_variants_in_file(figma_data):

    variant_count = 0


    def count_variants(node):
        nonlocal variant_count

        if node.get("type") == "COMPONENT_SET":
            variant_count += len(node.get("children", []))

        for child in node.get("children", []):
            count_variants(child)


    count_variants(figma_data.get("document"))
    return variant_count

In [7]:
variant_count = count_variants_in_file(figma_data)
print(f"Total number of variants in the file: {variant_count}")

Total number of variants in the file: 6706


In [ ]:
def save_raw_figma_data_to_json(figma_data, file_path):
    with open(file_path, 'w') as f:
        f.write(json.dumps(figma_data, indent=4))

In [ ]:
raw_figma_data_path = '/content/drive/My Drive/FYP content/Final thesis/raw_figma_data.json'

In [ ]:
save_raw_figma_data_to_json(figma_data, raw_figma_data_path)

## **Data extraction**


In [ ]:
def extract_componentset_name(node_name):
    # format is "style/component_name/subtype"
    parts = node_name.split('/')
    if len(parts) == 3:
        style, component_name, subtype = parts
    else:
        style, component_name, subtype = 'Unknown', 'Unknown', 'Unknown'
    return style.strip(), component_name.strip(), subtype.strip()


In [ ]:
def extract_variant_name(variant_name):
  #seperate the key value pairs for variants
    variant_details = {}
    pairs = variant_name.split(',')
    for pair in pairs:
        key, value = pair.split('=') if '=' in pair else ('Unknown', 'Unknown')
        key = key.strip()
        value = value.strip()
        if key not in variant_details:
            variant_details[key] = [value]
        else:
            variant_details[key].append(value)
    return variant_details if variant_details else {'state': ['default']}

In [ ]:
def process_variants(node, style, component_name, subtype):
  #looking into the variants and adding the commponent set details to it
    variants = []
    if node['type'] == 'COMPONENT_SET' or node['type'] == 'COMPONENT':
        variant_nodes = node['children'] if node['type'] == 'COMPONENT_SET' else [node]
        for child in variant_nodes:
            variant_details = extract_variant_name(child['name'])
            variant_properties = extract_properties(child)
            variant_properties.update({
                "style": style,
                "component_name": component_name,
                "subtype": subtype,
                "variant_details": variant_details,
            })
            variants.append({"variant_properties": variant_properties})
    return variants

In [ ]:
def extract_properties(node):
  #properties to extract
    properties = {
        'color': [],
        'strokes': [],
        'strokeWeight': 0,
        'text': [],
        'textColor': [],
        'borderRadius': [],
        'fontFamily': [],
        'fontWeight': [],
        'fontSize': [],
        'effects': [],
        'padding': 0,
        'width': 0,
        'height': 0,
        'x': 0,
        'y': 0
    }
    property_extraction(node, properties)
    return properties

In [ ]:
def property_extraction(node, properties):
    # to get colors
    fills = node.get('fills', [])
    for fill in fills:
        if fill['type'] == 'SOLID':
            color = fill['color']
            rgba = f"rgba({int(color['r'] * 255)}, {int(color['g'] * 255)}, {int(color['b'] * 255)}, {color['a']})"
            properties['color'] = rgba  # Change from append to direct assignment for simplified structure

    # to get cornerRadius
    if 'cornerRadius' in node:
        properties['borderRadius'] = node['cornerRadius']

    # Extract text properties
    if node['type'] == 'TEXT':
        properties['text'] = node.get('characters', '')
        textStyle = node.get('style', {})
        properties['fontFamily'] = textStyle.get('fontFamily', '')
        properties['fontWeight'] = textStyle.get('fontWeight', '')
        properties['fontSize'] = textStyle.get('fontSize', '')
        textFills = node.get('fills', [])
        for fill in textFills:
            if fill['type'] == 'SOLID':
                color = fill['color']
                rgba = f"rgba({int(color['r'] * 255)}, {int(color['g'] * 255)}, {int(color['b'] * 255)}, {color['a']})"
                properties['textColor'] = rgba

    # to get dimensions
    if 'absoluteBoundingBox' in node:
        boundingBox = node['absoluteBoundingBox']
        properties['width'] = boundingBox['width']
        properties['height'] = boundingBox['height']
        properties['x'] = boundingBox['x']
        properties['y'] = boundingBox['y']


    #to get strokes
    strokes = node.get('strokes', [])
    for stroke in strokes:
        if stroke['type'] == 'SOLID':
            color = stroke['color']
            rgba = f"rgba({int(color['r'] * 255)}, {int(color['g'] * 255)}, {int(color['b'] * 255)}, {color['a']})"
            if 'strokes' not in properties:  # Initialize if not exist
                properties['strokes'] = []
            properties['strokes'].append(rgba)
    strokeWeight = node.get('strokeWeight', 0)
    if strokeWeight:
        properties['strokeWeight'] = strokeWeight

    # to get effects
    effects = node.get('effects', [])
    for effect in effects:
        color = effect.get('color', {})
        rgba = f"rgba({int(color.get('r', 0) * 255)}, {int(color.get('g', 0) * 255)}, {int(color.get('b', 0) * 255)}, {color.get('a', 1)})"
        properties['effects'].append({'type': effect['type'], 'color': rgba})

    if 'padding' in node:
        properties['padding'] = node['padding']

    # go through child nodes
    if 'children' in node:
        for child in node['children']:
            property_extraction(child, properties)

### **Getting an output**

In [ ]:
def main_output(figma_data):
  #get output for button and input field
    component_types = ['button', 'input-field']
    organized_data = output_data(figma_data, component_types)


    print_output(organized_data, component_types)

In [ ]:
def output_data(file_data, component_types):
    organized_data = []
    #go into variants
    for canvas in file_data.get("document", {}).get("children", []):
        for node in canvas.get("children", []):
            if node["type"] in ["COMPONENT_SET", "COMPONENT"]:
                style, component_name, subtype = extract_componentset_name(node["name"])
                if component_name.lower() in component_types:
                    variants = process_variants(node, style, component_name, subtype)
                    organized_data.extend(variants)
    return organized_data

In [ ]:
def print_output(data, component_types):
    for comp_type in component_types:
        samples = [d for d in data if d["variant_properties"]["component_name"].lower() == comp_type][:2]
        for sample in samples:
            print(json.dumps(sample, indent=4))

In [ ]:
main_output(figma_data)

{
    "variant_properties": {
        "color": "rgba(255, 255, 255, 1.0)",
        "strokes": [
            "rgba(126, 86, 216, 1.0)"
        ],
        "strokeWeight": 1.0,
        "text": "Button CTA",
        "textColor": "rgba(255, 255, 255, 1.0)",
        "borderRadius": 10.0,
        "fontFamily": "Inter",
        "fontWeight": 500,
        "fontSize": 14.0,
        "effects": [
            {
                "type": "DROP_SHADOW",
                "color": "rgba(16, 24, 40, 0.05000000074505806)"
            }
        ],
        "padding": 0,
        "width": 77.0,
        "height": 20.0,
        "x": -4619.0,
        "y": -2135.0,
        "style": "Professional",
        "component_name": "Button",
        "subtype": "Default",
        "variant_details": {
            "State": [
                "Default"
            ],
            "Size": [
                "Small"
            ]
        }
    }
}
{
    "variant_properties": {
        "color": "rgba(255, 255, 255, 1.0)",
        "st

-------------------------------
\

## **Save data**

In [ ]:
def process_all_figma_data(file_data):
    #all components
    component_types = [
        'button', 'input-field', 'menu', 'menu-list', 'list-item',
        'card', 'label', 'iconbutton', 'search-field'
    ]

    component_types = [ctype.replace(" ", "").lower() for ctype in component_types]
    return output_data(file_data, component_types)

In [ ]:
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as f:
        for entry in data:

            compact_json = json.dumps(entry, separators=(',', ':'))
            f.write(compact_json + '\n')


In [ ]:
extracted_file_path = '/content/drive/My Drive/FYP content/Final thesis/components.jsonl'

In [ ]:
processed_data = process_all_figma_data(figma_data)

In [ ]:
save_to_jsonl(processed_data, extracted_file_path)

In [ ]:
# def load_from_jsonl(file_path):
#     data = []
#     with open(file_path, 'r') as f:
#         for line in f:
#             try:

#                 data.append(json.loads(line))
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding JSON: {e} in line: {line}")
#     return data


In [ ]:
# loaded_data = load_from_jsonl(extracted_file_path)